In [1]:
from pyspark.sql import SparkSession
# import pyspark.sql.functions as F
from pyspark.sql.functions import col, isnan, when, count, udf, to_date, year, month, date_format, size, split, datediff, regexp_extract, lit, abs
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, MinMaxScaler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
cleaned_data_path = 'gs://my-bigdata-project-rn/cleaned/cleaned_itineraries.parquet'
#cleaned_data_path = '/Users/robertn/Desktop/2024Fall/CIS4130/cleaned_itineraries.parquet'
#Sample used to collect only 5% of random data. Will be adjusted
sdf = spark.read.parquet(cleaned_data_path, header=True, inferSchema=True)

In [5]:
#Ensure search and flight date are datetime values
sdf.printSchema()

root
 |-- searchDate: date (nullable = true)
 |-- flightDate: date (nullable = true)
 |-- startingAirport: string (nullable = true)
 |-- destinationAirport: string (nullable = true)
 |-- travelDuration: string (nullable = true)
 |-- elapsedDays: integer (nullable = true)
 |-- isBasicEconomy: boolean (nullable = true)
 |-- isNonStop: boolean (nullable = true)
 |-- totalFare: double (nullable = true)
 |-- seatsRemaining: integer (nullable = true)
 |-- totalTravelDistance: integer (nullable = true)
 |-- segmentsArrivalAirportCode: string (nullable = true)
 |-- segmentsDepartureAirportCode: string (nullable = true)
 |-- segmentsAirlineCode: string (nullable = true)
 |-- segmentsEquipmentDescription: string (nullable = true)
 |-- segmentsDurationInSeconds: string (nullable = true)
 |-- segmentsDistance: string (nullable = true)
 |-- segmentsCabinCode: string (nullable = true)



In [3]:
# Engineer additional date feature columns based on the order_date
# Goal is to have a flightDate_OnWeekend column
sdf = sdf.withColumn("flightDate_DayOfWeek", date_format(col("flightDate"), "EEEE"))         # 'Monday' 'Tuesday' etc.
sdf = sdf.withColumn("flightDate_OnWeekend", when(sdf.flightDate_DayOfWeek == 'Saturday',1.0).when(sdf.flightDate_DayOfWeek == 'Sunday', 1.0).otherwise(0).cast('int'))

# Check columns to see if we got good values
sdf.select(['flightDate','flightDate_DayOfWeek', 'flightDate_OnWeekend']).show(10)

+----------+--------------------+--------------------+
|flightDate|flightDate_DayOfWeek|flightDate_OnWeekend|
+----------+--------------------+--------------------+
|2022-07-01|              Friday|                   0|
|2022-07-01|              Friday|                   0|
|2022-07-01|              Friday|                   0|
|2022-07-01|              Friday|                   0|
|2022-07-01|              Friday|                   0|
|2022-07-01|              Friday|                   0|
|2022-07-01|              Friday|                   0|
|2022-07-01|              Friday|                   0|
|2022-07-01|              Friday|                   0|
|2022-07-01|              Friday|                   0|
+----------+--------------------+--------------------+
only showing top 10 rows



In [4]:
#Create daysBetweenFlight column
sdf = sdf.withColumn("daysBetweenFlight", datediff(sdf.flightDate, sdf.searchDate))

In [5]:
#Create total travel duration column in a proper format
sdf = sdf.withColumn("travelduration_hours", regexp_extract(col("travelDuration"), r"(\d+)H",1).cast("int"))
sdf = sdf.withColumn("travelduration_minutes", regexp_extract(col("travelDuration"), r"(\d+)M",1).cast("int"))
sdf = sdf.na.fill(value=0.0,subset=["travelduration_minutes", "travelduration_hours"])
sdf = sdf.withColumn("travelduration_total_minutes", col("travelduration_hours") * 60 + col("travelduration_minutes"))

In [6]:
#Mapping true/false columns to 1 and 0
sdf = sdf.withColumn('isBasicEconomy', when(sdf.isBasicEconomy == 'true', 1).otherwise(0))
sdf = sdf.withColumn('isNonStop', when(sdf.isNonStop == 'true', 1).otherwise(0))

In [7]:
#Remove outlier elapsedDays values greater than 1
sdf = sdf.filter(col('elapsedDays') < 2)

In [8]:
#Verify no more outliers exist
sdf.groupBy('elapsedDays').count().show()

+-----------+--------+
|elapsedDays|   count|
+-----------+--------+
|          1|10943554|
|          0|63810675|
+-----------+--------+



In [9]:
#Apply MinMax to TotalTravelDistance
sdf = sdf.withColumn('totalTravelDistance', sdf.totalTravelDistance.cast('double'))
distance_assembler = VectorAssembler(inputCols=['totalTravelDistance'], outputCol='distanceVector')
totalDistance_scaler = MinMaxScaler(inputCol = 'distanceVector', outputCol = 'totalDistanceScaled')

#Indexer for string columns
indexer = StringIndexer(inputCols=['startingAirport', 'destinationAirport', 'segmentsArrivalAirportCode', 'segmentsCabinCode'], 
                        outputCols=['startAirportIndex', 'destAirportIndex', 'arrivalAirportCodeIndex', 'cabinCodeIndex'], handleInvalid="keep")

#One-Hot encoder
encoder = OneHotEncoder(inputCols=['startAirportIndex', 'destAirportIndex', 'arrivalAirportCodeIndex', 'cabinCodeIndex'],
                        outputCols=['startAirportVector', 'destAirportVector', 'arrivalAirportCodeVector', 'cabinCodeVector'],
                        dropLast=False)

#Creating the assembler for the feature vectors and integer columns used
assembler = VectorAssembler(inputCols=['startAirportVector', 'destAirportVector', 'arrivalAirportCodeVector', 'cabinCodeVector','daysBetweenFlight', 'flightDate_OnWeekend','travelduration_total_minutes', 'elapsedDays', 'isBasicEconomy', 'isNonStop', 'seatsRemaining', 'totalDistanceScaled'], outputCol='features')

# Create a Ridge Regression Estimator
ridge_reg = LinearRegression(labelCol='totalFare',  elasticNetParam=0, regParam=0.1)

# Create a regression evaluator (to get RMSE, R2, RME, etc.)
evaluator = RegressionEvaluator(labelCol='totalFare')

#Creating the pipeline

regression_pipe = Pipeline(stages=[indexer, encoder, distance_assembler, totalDistance_scaler, assembler, ridge_reg])

# Call .fit to transform the data
transformed_sdf = regression_pipe.fit(sdf).transform(sdf)

# Review the transformed features
print("Transformed features")
transformed_sdf.select('flightDate_OnWeekend', 'daysBetweenFlight','startingAirport','destinationAirport', 'travelduration_total_minutes', 'elapsedDays', 'isBasicEconomy', 'isNonStop', 'seatsRemaining', 'totalTravelDistance', 'segmentsArrivalAirportCode', 'segmentsCabinCode', 'totalFare', 'features').show(30, truncate=False)

24/12/13 03:06:35 WARN DAGScheduler: Broadcasting large task binary with size 1837.8 KiB
24/12/13 03:07:43 WARN DAGScheduler: Broadcasting large task binary with size 1839.0 KiB
24/12/13 03:07:45 WARN DAGScheduler: Broadcasting large task binary with size 1838.3 KiB
24/12/13 03:09:11 WARN DAGScheduler: Broadcasting large task binary with size 1839.5 KiB
24/12/13 03:09:11 WARN DAGScheduler: Broadcasting large task binary with size 1838.3 KiB
24/12/13 03:09:12 WARN DAGScheduler: Broadcasting large task binary with size 1839.5 KiB
24/12/13 03:09:13 WARN DAGScheduler: Broadcasting large task binary with size 1838.3 KiB
24/12/13 03:09:14 WARN DAGScheduler: Broadcasting large task binary with size 1839.5 KiB
24/12/13 03:09:14 WARN DAGScheduler: Broadcasting large task binary with size 1838.3 KiB
24/12/13 03:09:15 WARN DAGScheduler: Broadcasting large task binary with size 1839.5 KiB
24/12/13 03:09:16 WARN DAGScheduler: Broadcasting large task binary with size 1838.3 KiB
24/12/13 03:09:16 WAR

Transformed features
+--------------------+-----------------+---------------+------------------+----------------------------+-----------+--------------+---------+--------------+-------------------+--------------------------+-------------------+---------+---------------------------------------------------------------------------------------------------------------+
|flightDate_OnWeekend|daysBetweenFlight|startingAirport|destinationAirport|travelduration_total_minutes|elapsedDays|isBasicEconomy|isNonStop|seatsRemaining|totalTravelDistance|segmentsArrivalAirportCode|segmentsCabinCode  |totalFare|features                                                                                                       |
+--------------------+-----------------+---------------+------------------+----------------------------+-----------+--------------+---------+--------------+-------------------+--------------------------+-------------------+---------+------------------------------------------------------

In [10]:
# Split the data into 70% training and 30% test sets  
trainingData, testData = sdf.randomSplit([0.7, 0.3], seed=42)

# Create a grid to hold hyperparameters 
grid = ParamGridBuilder()

# Build the parameter grid
grid = grid.build()

# Create the CrossValidator using the hyperparameter grid
cv = CrossValidator(estimator=regression_pipe, 
                    estimatorParamMaps=grid, 
                    evaluator=evaluator, 
                    numFolds=3)

# Train the models
all_models  = cv.fit(trainingData)

# Show the average performance over the three folds
print(f"Average metric {all_models.avgMetrics}")

# Get the best model from all of the models trained
bestModel = all_models.bestModel

# Use the model 'bestModel' to predict the test set
test_results = bestModel.transform(testData)
# Show the predicted tip
test_results.select('flightDate_OnWeekend', 'daysBetweenFlight','startingAirport','destinationAirport', 'travelduration_total_minutes', 'elapsedDays', 'isBasicEconomy', 'isNonStop', 'seatsRemaining', 'totalTravelDistance', 'segmentsArrivalAirportCode', 'segmentsCabinCode', 'totalFare', 'prediction').show(truncate=False)
# Calculate RMSE and R2
rmse = evaluator.evaluate(test_results, {evaluator.metricName:'rmse'})
r2 =evaluator.evaluate(test_results,{evaluator.metricName:'r2'})
print(f"RMSE: {rmse}  R-squared:{r2}")

24/12/13 03:12:06 WARN DAGScheduler: Broadcasting large task binary with size 1748.3 KiB
24/12/13 03:12:30 WARN DAGScheduler: Broadcasting large task binary with size 1749.6 KiB
24/12/13 03:12:30 WARN DAGScheduler: Broadcasting large task binary with size 1748.8 KiB
24/12/13 03:12:56 WARN DAGScheduler: Broadcasting large task binary with size 1750.0 KiB
24/12/13 03:12:56 WARN DAGScheduler: Broadcasting large task binary with size 1748.8 KiB
24/12/13 03:12:56 WARN DAGScheduler: Broadcasting large task binary with size 1750.0 KiB
24/12/13 03:12:56 WARN DAGScheduler: Broadcasting large task binary with size 1748.8 KiB
24/12/13 03:12:57 WARN DAGScheduler: Broadcasting large task binary with size 1750.0 KiB
24/12/13 03:12:57 WARN DAGScheduler: Broadcasting large task binary with size 1748.8 KiB
24/12/13 03:12:57 WARN DAGScheduler: Broadcasting large task binary with size 1750.0 KiB
24/12/13 03:12:57 WARN DAGScheduler: Broadcasting large task binary with size 1748.8 KiB
24/12/13 03:12:58 WAR

Average metric [126.5802749971769]


24/12/13 03:27:01 WARN DAGScheduler: Broadcasting large task binary with size 1855.3 KiB


+--------------------+-----------------+---------------+------------------+----------------------------+-----------+--------------+---------+--------------+-------------------+--------------------------+-------------------+---------+------------------+
|flightDate_OnWeekend|daysBetweenFlight|startingAirport|destinationAirport|travelduration_total_minutes|elapsedDays|isBasicEconomy|isNonStop|seatsRemaining|totalTravelDistance|segmentsArrivalAirportCode|segmentsCabinCode  |totalFare|prediction        |
+--------------------+-----------------+---------------+------------------+----------------------------+-----------+--------------+---------+--------------+-------------------+--------------------------+-------------------+---------+------------------+
|1                   |40               |ORD            |MIA               |186                         |0          |0             |1        |2             |1192.0             |MIA                       |coach              |338.61   |230.2703

24/12/13 03:27:18 WARN DAGScheduler: Broadcasting large task binary with size 1861.8 KiB
24/12/13 03:28:01 WARN DAGScheduler: Broadcasting large task binary with size 1862.9 KiB
24/12/13 03:28:01 WARN DAGScheduler: Broadcasting large task binary with size 1861.8 KiB


RMSE: 126.42190553009543  R-squared:0.5888453561622946


24/12/13 03:28:44 WARN DAGScheduler: Broadcasting large task binary with size 1862.9 KiB


In [ ]:
print(bestModel.stages)

coefficients = bestModel.stages[5].coefficients
# print("bestModel coefficients", coefficients)
intercept = bestModel.stages[5].intercept
print("bestModel intercept", intercept)

#Finding the most important features
feature_names = ['daysBetweenFlight','flightDate_OnWeekend','startingAirport','destinationAirport', 'travelduration_total_minutes', 'elapsedDays', 'isBasicEconomy', 'isNonStop', 'seatsRemaining', 'totalTravelDistance', 'segmentsArrivalAirportCode', 'segmentsCabinCode']

coef_map = dict(zip(feature_names, coefficients))

print("Feature Importance:")
for feature, coef in coef_map.items():
    print("  {}: {:.3f}".format(feature, coef))

In [ ]:
transformed_sdf.write.parquet('gs://my-bigdata-project-rn/trusted/trusted_ridge_itineraries.parquet')

24/12/13 03:28:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/13 03:28:45 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


In [ ]:
model_path =  'gs://my-bigdata-project-rn/models/plane_ridge_regression_model'
bestModel.write().overwrite().save(model_path)